<a href="https://colab.research.google.com/github/dmitryglhf/jupyter-projects/blob/main/lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data

In [38]:
import numpy as np
import pandas as pd

In [39]:
#df = pd.read_csv('our data...')

### QR-algorithm


In [40]:
def householder(a):
    v = a.copy()
    v[0] += np.sign(v[0]) * np.linalg.norm(a)
    H = np.eye(len(a)) - 2 * np.outer(v, v) / np.dot(v, v)  # getting Householder matrix
    return H


def get_qr(A):
    n = A.shape[0]
    Q = np.eye(n)
    R = A.copy()

    for j in range(n):
        a = R[j:, j]  # getting vector
        H = householder(a)  # getting Householder matrix

        R[j:, j:] = H @ R[j:, j:]  # change R
        Q[:, j:] = Q[:, j:] @ H.T  # change Q

    return Q, R


def eig(A, e=1e-10):
    An = A.copy() # for eig values
    eig_vectors = np.eye(A.shape[0]) # for eig vectors
    cache_eig_values = np.diag(An) # diag values
    diff = 1000

    while diff > e:
        Q, R = get_qr(An)

        # update eigen values and eigen vectors
        An = R @ Q
        eig_vectors = eig_vectors @ Q

        # update difference between eig values
        current_eig_values = np.diag(An)
        diff = np.linalg.norm(current_eig_values - cache_eig_values)
        cache_eig_values = current_eig_values

    eig_values = np.diag(An)

    # sort eig_values
    sorted_key = np.argsort(eig_values)[::-1]
    eig_values = eig_values[sorted_key]

    # return eigen values and eigen vectors
    return eig_values, eig_vectors

### SVD

In [45]:
def svd(A):
    # V^T matrix
    temp_V = A.T @ A
    eig_values, V = eig(temp_V)  # Собственные значения и векторы для A^T A
    V_T = V.T

    # U matrix
    temp_U = A @ A.T
    _, U = eig(temp_U)  # Собственные значения и векторы для A A^T

    # Sigma matrix ПРОВЕРИТЬ ЭТОТ МОМЕНТ
    sigma = np.zeros((A.shape[0], A.shape[1]))
    for i in range(min(A.shape[0], A.shape[1])):
        sigma[i, i] = np.sqrt(eig_values[i]) if eig_values[i] >= 0 else 0

    # Result
    return U, sigma, V_T

### Data Scaling before PCA??

In [ ]:
# before pca
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_X = scaler.fit_transform(df)
scaled_X.mean()

In [ ]:
cov_matrix = np.cov(scaled_X, rowvar = False)

### PCA

In [ ]:
# coming soon...

### Build the Neural Network

https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html#model-parameters

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

#### Get Device for Training

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

#### Define the Class

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork().to(device)
print(model)

In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

### Test - Eigen values and vectors

In [32]:
'''
eigen values for matrix A:
l1 = -2
l2 = 2
l3 = 4

eigen vectors for matrix A:
v1 = (0, 1, -2)
v2 = (2, -1, 0)
v3 = (9, -2, 1)

original: http://mathprofi.ru/sobstvennye_znachenija_i_sobstvennye_vektory.html
'''

A = np.array([[5, 6, 3],
                      [-1, 0, 1],
                      [1, 2, -1]], dtype = float)
A=A.T@A
print(f'Transposed origin matrix A: \n {A} \n ')
eig_values, eig_vectors = eig(A)

n = A.shape[0]

for i in range(n):
    v = eig_vectors [:, i]
    norm_value = np.linalg.norm((A - eig_values[i] * np.eye(n)) @ v)

    print(f"Норма для собственного значения \
    λ{i+1} = {eig_values[i].round(3)}:\t {norm_value.round(3)}")

print(f'\n Eigen vectors: \n {eig_vectors}')

Transposed origin matrix A: 
 [[27. 32. 13.]
 [32. 40. 16.]
 [13. 16. 11.]] 
 
Норма для собственного значения     λ1 = 73.007:	 0.0
Норма для собственного значения     λ2 = 4.147:	 0.0
Норма для собственного значения     λ3 = 0.846:	 0.0

 Eigen vectors: 
 [[ 0.60030706 -0.21284941  0.77092578]
 [ 0.73491404 -0.23343806 -0.6367166 ]
 [ 0.31548817  0.94878965  0.01629151]]


### Test - SVD-decomposition

In [46]:
'''
Для матрицы A_2x2:

U = [[0.6, 0.8]
      [0.8, -0.6]]

Sigma = [[3, 0]
              [0, 1]]

V_T = [[0.8, -0.6]
          [0.6, 0.8]]

original: http://www.machinelearning.ru/wiki/index.php?title=%D0%A1%D0%B8%D0%BD%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%BD%D0%BE%D0%B5_%D1%80%D0%B0%D0%B7%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5
'''

A_2x2 = np.array([[0.96, 1.72],
                            [2.28, 0.96]], dtype = float)

U, Sigma, V_T = svd(A_2x2)

print(f'Origin matrix A: \n{A_2x2} \n ')
print(f'Матрица левых сингулярных векторов U: \n {U.round(3)} \n ')
print(f'Матрица сингулярных чисел Sigma: \n {Sigma.round(3)} \n ')
print(f'Матрица правых сингулярных векторов V_T: \n {V_T.round(3)} ')

Origin matrix A: 
[[0.96 1.72]
 [2.28 0.96]] 
 
Матрица левых сингулярных векторов U: 
 [[-0.6  0.8]
 [-0.8 -0.6]] 
 
Матрица сингулярных чисел Sigma: 
 [[3. 0.]
 [0. 1.]] 
 
Матрица правых сингулярных векторов V_T: 
 [[-0.8 -0.6]
 [ 0.6 -0.8]] 


### Комментарии


#### Заира

Делаем так: нам нужно найти разложение X вида $U*Сигма*V_T$:
- находим собственные значения для матрицы A*A_T и собств вектора -> определим U и Сигму из этого

Cейчас (до вс) нужно научиться сортировать в сигме синг значения по убыванию и менять соответсвующие столбцы в U.
Что делать с матрицей V я подумаю и скажу в вс

#### Дима

__Сделать__

- добить свд и посмотреть что в нем с матрицей сигма и на кой там внутри сортировка теперь нужна, если она есть в eig()
- протестировать свд на примере (мб презентация)
- что с проверкой матрицы на симметричность и нужно ли внутри метода умножать на транспонированную чтобы сделать симметричной

#### Вадим

можете оставлять здесь комментарии, заметки и прикладывать картинки.

тыкни два раза по этому тексту чтобы редактировать его

#### Алена

можете оставлять здесь комментарии, заметки и прикладывать картинки.

тыкни два раза по этому тексту чтобы редактировать его